In [1]:
# A_Aug_Apply_BG_Fill_For_Flag_Class
'''
Include in dissertation
This script refines datasets containing Green Coast flags by removing incorrectly hanging red/yellow flags. 
Bounding boxes for a specified class (`class_to_fill`) are filled with sampled background colors, 
while protecting regions corresponding to another specified class (`class_to_keep`). The resulting images 
and updated YOLO-format annotation files are saved to the target directory.

## Workflow:
1. Identify Bounding Boxes:
   - Reads YOLO-format annotations to locate the bounding boxes for the `class_to_fill` and `class_to_keep`.
2. Background Sampling and Filling:
   - Samples surrounding areas (above, left, right) to calculate a fill color for the `class_to_fill` regions.
   - Excludes overlapping regions for `class_to_keep` during filling.
3. Save Results
   - Saves the modified images with filled regions.
   - Updates and saves the corresponding YOLO-format annotation files.

## Parameters:
- `src_folder` (str): Path to the folder containing source images and annotations.
- `tgt_folder` (str): Path to save the modified images and annotations.
- `class_to_fill` (int): The class ID of the bounding boxes to be removed and filled.
- `class_to_keep` (int): The class ID of the bounding boxes to be protected from filling.
- `padding` (tuple): Padding for bounding box adjustments (top, right, bottom, left).
- `x_offset` (int): Horizontal offset applied to the `class_to_fill` regions.

## Functions:
- `create_filled_filename`: Generates a new filename for processed images and annotations.
- `fill_bbox_with_background`: Fills bounding boxes for `class_to_fill` with sampled background color.
- `fill_class`: Processes all images and annotations in a source folder.

## Example:
1. Specify the paths to the source and target folders:
    - `src_folder = 'path/to/source/images'`
    - `tgt_folder = 'path/to/target/output'`
2. Define the classes to fill and protect:
    - `class_to_fill = 1` (e.g., red/yellow flags)
    - `class_to_keep = 3` (e.g., Green Coast flags)
3. Run the script to process all images and annotations:
    fill_class(src_folder, tgt_folder, class_to_fill, class_to_keep, padding=(5, 5, 0, 5), x_offset=30)

## Notes:
- Input images must be in `.png` format with corresponding `.txt` YOLO annotation files.
- Processed images will be saved with modified filenames indicating the filled class.
'''
    
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def create_filled_filename(file_name, source_class_id):
    """
    Create a new filename for the merged image or annotation based on user specifications.

    Args:
        file_name (str): The original filename.
        source_class_id (int): The class ID to be removed and filled in the images.

    Returns:
        str: A new filename that follows the specified naming convention.
    """
    base_name, ext = os.path.splitext(file_name)
    new_name = f"{base_name}_Fill_CLS_{source_class_id}{ext}"
    #new_name = file_name

    return new_name

def fill_bbox_with_background(tgt_image, tgt_annotations, target_class, keep_class_id, x_offset=0, padding=(10, 10, 10, 10)):
    """
    Fill the bounding box for the target class with the background colour sampled from surrounding areas,
    excluding areas that overlap with the keep class bounding boxes.

    Args:
        tgt_image (ndarray): The target image.
        tgt_annotations (list): List of annotations (YOLO format).
        target_class (int): The class ID of the target bounding box to be filled.
        keep_class_id (int): The class ID of the bounding box to be excluded from the filling area.
        padding (tuple): Padding for (top, right, bottom, left) directions.
        x_offset (int): Horizontal offset applied to the bounding box.

    Returns:
        tgt_image (ndarray): The updated image with the target bounding box filled.
    """
    top_pad, right_pad, bottom_pad, left_pad = padding
    image_height, image_width = tgt_image.shape[:2]
    target_bbox = None

    # Create an exclusion mask for the keep class
    exclusion_mask = np.zeros((image_height, image_width), dtype=np.uint8)
    for annotation in tgt_annotations:
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == keep_class_id:
            x_min = int((x_center - width / 2) * image_width)
            y_min = int((y_center - height / 2) * image_height)
            x_max = int((x_center + width / 2) * image_width)
            y_max = int((y_center + height / 2) * image_height)
            cv2.rectangle(exclusion_mask, (x_min, y_min), (x_max, y_max), 255, thickness=-1)

    for annotation in tgt_annotations:
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == target_class:
            # Apply x_offset to background fill
            x_min = int((x_center - width / 2) * image_width) + x_offset
            y_min = int((y_center - height / 2) * image_height)
            x_max = int((x_center + width / 2) * image_width) + x_offset
            y_max = int((y_center + height / 2) * image_height)

            # Ensure coordinates are within bounds
            x_min = max(0, x_min - left_pad)
            y_min = max(0, y_min - top_pad)
            x_max = min(image_width, x_max + right_pad)
            y_max = min(image_height, y_max + bottom_pad)

            # Exclude overlapping areas with keep class
            target_mask = np.zeros((image_height, image_width), dtype=np.uint8)
            cv2.rectangle(target_mask, (x_min, y_min), (x_max, y_max), 255, thickness=-1)
            target_mask = cv2.bitwise_and(target_mask, cv2.bitwise_not(exclusion_mask))

            # Sample regions outside the bounding box
            top = tgt_image[max(0, y_min - top_pad):y_min, x_min:x_max]
            left = tgt_image[y_min:y_max, max(0, x_min - left_pad):x_min]
            right = tgt_image[y_min:y_max, x_max:min(image_width, x_max + right_pad)]

            # Combine the mean colours from sampled regions
            sampled_colors = []
            if top.size > 0:
                sampled_colors.append(top.mean(axis=(0, 1)))
            if left.size > 0:
                sampled_colors.append(left.mean(axis=(0, 1)))
            if right.size > 0:
                sampled_colors.append(right.mean(axis=(0, 1)))

            if not sampled_colors:
                print("Sampling failed for bounding box. Skipping background filling.")
                continue

            # Calculate the average colour from all sampled regions
            background_color = np.mean(sampled_colors, axis=0).astype(np.uint8)

            # Fill the target mask area with the computed background colour
            tgt_image[target_mask > 0] = background_color
            target_bbox = (x_min, y_min, x_max, y_max)
            break

    if target_bbox is None:
        return
    return tgt_image

def fill_bbox_with_backgroundWORKING(tgt_image, tgt_annotations, target_class, keep_class_id, x_offset=0, padding=(10, 10, 10, 10)):
    """
    Fill the bounding box for the target class with the background colour sampled from surrounding areas,
    excluding areas that overlap with the keep class bounding boxes.

    Args:
        tgt_image (ndarray): The target image.
        tgt_annotations (list): List of annotations (YOLO format).
        target_class (int): The class ID of the target bounding box to be filled.
        keep_class_id (int): The class ID of the bounding box to be excluded from the filling area.
        padding (tuple): Padding for (top, right, bottom, left) directions.
        x_offset (int): Horizontal offset applied to the bounding box.

    Returns:
        tgt_image (ndarray): The updated image with the target bounding box filled.
    """
    top_pad, right_pad, bottom_pad, left_pad = padding
    image_height, image_width = tgt_image.shape[:2]
    target_bbox = None

    # Create an exclusion mask for the keep class
    exclusion_mask = np.zeros((image_height, image_width), dtype=np.uint8)
    for annotation in tgt_annotations:
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == keep_class_id:
            x_min = int((x_center - width / 2) * image_width)
            y_min = int((y_center - height / 2) * image_height)
            x_max = int((x_center + width / 2) * image_width)
            y_max = int((y_center + height / 2) * image_height)
            cv2.rectangle(exclusion_mask, (x_min, y_min), (x_max, y_max), 255, thickness=-1)

    for annotation in tgt_annotations:
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == target_class:
            # Apply x_offset to background fill
            x_min = int((x_center - width / 2) * image_width) + x_offset
            y_min = int((y_center - height / 2) * image_height)
            x_max = int((x_center + width / 2) * image_width) + x_offset
            y_max = int((y_center + height / 2) * image_height)

            # Ensure coordinates are within bounds
            x_min = max(0, x_min - left_pad)
            y_min = max(0, y_min - top_pad)
            x_max = min(image_width, x_max + right_pad)
            y_max = min(image_height, y_max + bottom_pad)

            # Exclude overlapping areas with keep class
            target_mask = np.zeros((image_height, image_width), dtype=np.uint8)
            cv2.rectangle(target_mask, (x_min, y_min), (x_max, y_max), 255, thickness=-1)
            target_mask = cv2.bitwise_and(target_mask, cv2.bitwise_not(exclusion_mask))

            # Sample regions outside the bounding box
            top = tgt_image[max(0, y_min - top_pad):y_min, x_min:x_max]
            left = tgt_image[y_min:y_max, max(0, x_min - left_pad):x_min]
            right = tgt_image[y_min:y_max, x_max:min(image_width, x_max + right_pad)]

            # Combine the mean colours from sampled regions
            sampled_colors = []
            if top.size > 0:
                sampled_colors.append(top.mean(axis=(0, 1)))
            if left.size > 0:
                sampled_colors.append(left.mean(axis=(0, 1)))
            if right.size > 0:
                sampled_colors.append(right.mean(axis=(0, 1)))

            if not sampled_colors:
                print("Sampling failed for bounding box. Skipping background filling.")
                continue

            # Calculate the average colour from all sampled regions
            background_color = np.mean(sampled_colors, axis=0).astype(np.uint8)

            # Fill the target mask area with the computed background colour
            tgt_image[target_mask > 0] = background_color
            target_bbox = (x_min, y_min, x_max, y_max)
            break

    if target_bbox is None:
        return
    return tgt_image


def fill_class(src_folder, tgt_folder, source_class_id, keep_class_id, scale_factor=1.0, padding=(10, 10, 10, 10), x_offset=0):
    if not os.path.exists(tgt_folder):
        os.makedirs(tgt_folder)
    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith('.png'):
            src_image = cv2.imread(os.path.join(src_folder, file_name))
            src_annotation_path = os.path.join(src_folder, os.path.splitext(file_name)[0] + ".txt")
            if not os.path.exists(src_annotation_path):
                print(f"No annotations for {file_name} in source ({src_folder}). Skipping.")
                continue
            with open(src_annotation_path, 'r') as f:
                src_annotations = [list(map(float, line.split())) for line in f]

            # Fill the class with the background colour samples, excluding keep_class_id
            background_filled_image = fill_bbox_with_background(
                tgt_image=src_image,
                tgt_annotations=src_annotations,
                target_class=source_class_id,
                keep_class_id=keep_class_id,
                x_offset=x_offset,
                padding=padding
            )

            # Remove the annotations of the target class
            updated_annotations = [
                annotation for annotation in src_annotations
                if int(annotation[0]) != source_class_id
            ]

            # Save the image and annotation file
            if background_filled_image is not None:
                new_filename = create_filled_filename(file_name, source_class_id)
                output_path = os.path.join(tgt_folder, new_filename)
                cv2.imwrite(output_path, background_filled_image)

                # Save the updated annotations
                new_annotation_path = os.path.join(
                    tgt_folder, os.path.splitext(new_filename)[0] + ".txt"
                )
                with open(new_annotation_path, 'w') as f:
                    for annotation in updated_annotations:
                        annotation[0] = int(annotation[0])  # Ensure class ID is an integer
                        f.write(' '.join(f"{value:.6f}" if i > 0 else str(int(value)) for i, value in enumerate(annotation)) + '\n')

                
# Stage 1 - fill the red / yellow class bounding box 
#fill_class_in = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_52_illum'
#output = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_52_ClassFill'
#fill_class_in = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_51_illum'
#output = 'D:/FlagDetectionDatasets/ExportedDatasetsReduced/Job_51_ClassFill'

fill_class_in = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Merge_red_yellow_mask_from_class_1_in_30_to_24/000328' #Flag_To_Swap_From_30_Class_1'
output = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Merge_red_yellow_mask_from_class_1_in_30_to_24/000328_Filled2' #Flag_To_Swap_From_30_Class_2_filled'

class_to_fill = 2 # Fill blue flag 1  # Fill red / yellow flag class 
class_to_keep = 1 # red yellow 3  # Green coast flag to protect 
x_offset = 0   # 30 works well for set 51 - pointing to the right 
padding=(5, 5, 0, 10) #(5, 5, 0, 5)   # Padding: (top, right, bottom, left) #padding=(0, 5, 5, 5)

fill_class(fill_class_in, output, class_to_fill, class_to_keep, padding, x_offset=x_offset)

# Next run # Title: A_Aug_Replace_Class_Using_HSV_Red_Yellow_colour_threshold.jpynb 
